<a href="https://colab.research.google.com/github/gilvandrocesardemedeiros/ClimatePrediction/blob/master/Replace_Missing_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Importando bibliotecas
import pandas as pd
import numpy as np
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import np_utils
import keras.backend as tf
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
#Nomes para as colunas
columns = ["Estacao","Data","Hora","Prec","TempMax","TempMin","Insolacao","EvapoPiche","TempCompMedia",
           "UmidRelatMedia", "VelocVento", "Missing"]

#Tipos de dados em cada coluna
types = {"Hora":np.uint16,"Prec":np.float64,"TempMax":np.float64,"TempMin":np.float64,"Insolacao":np.float64,
         "EvapoPiche":np.float64,"TempCompMedia":np.float64,"UmidRelatMedia":np.float64,"VelocVento":np.float64}

#Leitura do arquivo csv
dataSet = pd.read_csv("https://github.com/gilvandrocesardemedeiros/ClimatePrediction/raw/master/Dados_INMET/Diarios/INMET-Dados_Diarios_Natal_1968-2018.csv",
                      sep=';',skiprows=48, nrows = 27003, usecols = columns[1:11], names=columns, dtype = types)

In [0]:
#Transformando data em variável do tipo datetime
dataSet["Data"] = pd.to_datetime(dataSet["Data"], format = "%d/%m/%Y")

In [6]:
#Verificando dados
dataSet.head()

,Data,Hora,Prec,TempMax,TempMin,Insolacao,EvapoPiche,TempCompMedia,UmidRelatMedia,VelocVento
0,1968-01-01,0,NaN,30.6,NaN,11.1,4.4,25.14,73.50,2.666667
1,1968-01-01,1200,0.0,NaN,21.4,NaN,NaN,NaN,NaN,NaN
2,1968-01-02,0,NaN,31.0,NaN,11.9,6.5,24.92,74.75,1.666667
3,1968-01-02,1200,0.0,NaN,19.6,NaN,NaN,NaN,NaN,NaN
4,1968-01-03,0,NaN,31.6,NaN,11.3,5.0,24.90,77.25,2.333333


In [7]:
#Verificando o tipo dos dados
dataSet.dtypes

Data              datetime64[ns]
Hora                      uint16
Prec                     float64
TempMax                  float64
TempMin                  float64
Insolacao                float64
EvapoPiche               float64
TempCompMedia            float64
UmidRelatMedia           float64
VelocVento               float64
dtype: object

In [0]:
#Separando os dados em dois dataframes, um para as 00:00 h e outro para as 12:00 h
dataSet00, dataSet12 = dataSet[dataSet["Hora"] == 0], dataSet[dataSet["Hora"] == 1200]

In [9]:
#Verificando dataframe para os dados disponibilizados às 00:00 h
dataSet00.head()

,Data,Hora,Prec,TempMax,TempMin,Insolacao,EvapoPiche,TempCompMedia,UmidRelatMedia,VelocVento
0,1968-01-01,0,NaN,30.6,NaN,11.1,4.4,25.14,73.50,2.666667
2,1968-01-02,0,NaN,31.0,NaN,11.9,6.5,24.92,74.75,1.666667
4,1968-01-03,0,NaN,31.6,NaN,11.3,5.0,24.90,77.25,2.333333
6,1968-01-04,0,NaN,31.2,NaN,10.3,4.0,24.82,80.75,3.666667
8,1968-01-05,0,NaN,30.8,NaN,11.0,4.2,24.78,76.00,4.000000


In [10]:
#Verificando dataframe para os dados disponibilizados às 12:00 h
dataSet12.head()

,Data,Hora,Prec,TempMax,TempMin,Insolacao,EvapoPiche,TempCompMedia,UmidRelatMedia,VelocVento
1,1968-01-01,1200,0.0,NaN,21.4,NaN,NaN,NaN,NaN,NaN
3,1968-01-02,1200,0.0,NaN,19.6,NaN,NaN,NaN,NaN,NaN
5,1968-01-03,1200,0.0,NaN,18.5,NaN,NaN,NaN,NaN,NaN
7,1968-01-04,1200,0.0,NaN,20.7,NaN,NaN,NaN,NaN,NaN
9,1968-01-05,1200,0.0,NaN,19.9,NaN,NaN,NaN,NaN,NaN


In [0]:
#Transformando a data no índice de cada dataframe
dataSet00 = dataSet00.set_index("Data")
dataSet12 = dataSet12.set_index("Data")

In [0]:
#Descartando a coluna "Hora"
dataSet00, dataSet12 = dataSet00.drop(columns = "Hora"), dataSet12.drop(columns = "Hora")

In [13]:
#Juntando os dados em um mesmo dataframe (no caso, o dataSet00)
for i in dataSet00.index:
    try:
        dataSet00["Prec"].loc[i] = dataSet12["Prec"].loc[i]
        dataSet00["TempMin"].loc[i] = dataSet12["TempMin"].loc[i]
    except:
        print("Data " + str(i.day) + "/" + str(i.month) + "/" + str(i.year) + " Não encontrada!")

Data 5/7/1986 Não encontrada!
Data 20/7/1986 Não encontrada!
Data 3/6/1989 Não encontrada!
Data 12/6/1989 Não encontrada!
Data 4/12/1992 Não encontrada!
Data 26/5/2018 Não encontrada!
Data 1/8/2018 Não encontrada!
Data 27/10/2018 Não encontrada!


In [14]:
#Atribuindo à variável dataSet o DataFrame atualizado
dataSet = dataSet00
#Verificando dimensões do DataFrame
print(dataSet.shape)

(13502, 8)


In [0]:
#Transformando a coluna Data de volta em uma coluna de informações do dataSet
dataSet = dataSet.reset_index()

In [16]:
#Eliminando da base de dados dias que não tenham ao menos 5 das variáveis no dataSet
dataSet = dataSet.dropna(thresh=5)
#Exibindo novas dimensões da base de dados
print(dataSet.shape)

(13455, 9)


# Considerando os seguintes intervalos: 
- Outono: 20 de março a 21 de junho (fechado em 20/03 e aberto em 21/06)
- Inverno: 21 de junho a 22 de setembro (fechado em 21/06 e aberto em 22/09)
- Primavera: de 22 de setembro a 21 de dezembro (fechado em 22/09 e aberto em 21/12)
- Verão: de 21 de dezembro a 20 de março (fechado em 21/12 e aberto em 20/03)

In [0]:
#Função que retorna a estação do ano para uma determinada data, para ser adicionada à base de dados
def season(data):
    if (data.month > 3 and data.month < 6) or (data.month == 3 and data.day >= 20) or (data.month == 6 and data.day < 21):
        return "Out"
    if (data.month > 6 and data.month < 9) or (data.month == 6 and data.day >= 21) or (data.month == 9 and data.day < 22):
        return "Inv"
    if (data.month > 9 and data.month < 12) or (data.month == 9 and data.day >= 22) or (data.month == 12 and data.day < 21):
        return "Pri"
    else:
        return "Ver"

In [0]:
#Identificando a estação do ano na base de dados e guardando a informação referente ao ano de cada dado
estacao = []
for i in dataSet.index:
    estacao.insert(i, season(dataSet.loc[i, "Data"]))

In [19]:
#Contando quantos dados existem para cada estação
print("Outono: " + str(estacao.count("Out")))
print("Inverno: " + str(estacao.count("Inv")))
print("Primavera: " + str(estacao.count("Pri")))
print("Verão: " + str(estacao.count("Ver")))

Outono: 3397
Inverno: 3460
Primavera: 3337
Verão: 3261


In [20]:
#Eliminando a coluna "Data" e visualizando dados
dataSet = dataSet.drop(columns="Data")
dataSet.head()

,Prec,TempMax,TempMin,Insolacao,EvapoPiche,TempCompMedia,UmidRelatMedia,VelocVento
0,0.0,30.6,21.4,11.1,4.4,25.14,73.50,2.666667
1,0.0,31.0,19.6,11.9,6.5,24.92,74.75,1.666667
2,0.0,31.6,18.5,11.3,5.0,24.90,77.25,2.333333
3,0.0,31.2,20.7,10.3,4.0,24.82,80.75,3.666667
4,0.0,30.8,19.9,11.0,4.2,24.78,76.00,4.000000


In [0]:
#Decodificando as estações em uma matriz (one hot encoding - dummy variables)
encoder = LabelEncoder()
encoder.fit(estacao)
encodedSeason = encoder.transform(estacao)
dummySeason = np_utils.to_categorical(encodedSeason)

In [22]:
#Identificar como aconteceu a transformação
auxSeason = encoder.transform(["Inv","Out","Pri","Ver"])
auxDummy = np_utils.to_categorical(auxSeason)
print("[Inv,Out,Pri,Ver]")
print(auxDummy)

[Inv,Out,Pri,Ver]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [0]:
#Separando a variável que será prevista, precipitação, da base de dados
prec = dataSet["Prec"]
dataSet = dataSet.drop(columns = "Prec")

In [0]:
#Adicionando ao dataSet as informações de estação climática
dataSet["Inv"] = dummySeason[:,0]
dataSet["Out"] = dummySeason[:,1]
dataSet["Pri"] = dummySeason[:,2]
dataSet["Ver"] = dummySeason[:,3]

In [25]:
#Visualizando dados
dataSet.head()

,TempMax,TempMin,Insolacao,EvapoPiche,TempCompMedia,UmidRelatMedia,VelocVento,Inv,Out,Pri,Ver
0,30.6,21.4,11.1,4.4,25.14,73.50,2.666667,0.0,0.0,0.0,1.0
1,31.0,19.6,11.9,6.5,24.92,74.75,1.666667,0.0,0.0,0.0,1.0
2,31.6,18.5,11.3,5.0,24.90,77.25,2.333333,0.0,0.0,0.0,1.0
3,31.2,20.7,10.3,4.0,24.82,80.75,3.666667,0.0,0.0,0.0,1.0
4,30.8,19.9,11.0,4.2,24.78,76.00,4.000000,0.0,0.0,0.0,1.0


In [0]:
#Efetuando uma transformação robusta, trabalhando com os dados entre os percentis 10 e 90
rs = RobustScaler(quantile_range = (10.0,90.0))
dataSet = rs.fit_transform(dataSet)

In [27]:
#Trocando dados faltantes por 0
np.nan_to_num(dataSet, copy = False)

array([[ 0.28571429, -0.31481481,  0.22666667, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.42857143, -0.64814815,  0.33333333, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.64285714, -0.85185185,  0.25333333, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.21428571, -0.33333333,  0.18666667, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.42857143,  0.57407407,  0.26666667, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.28571429,  0.44444444,  0.21333333, ...,  0.        ,
         0.        ,  1.        ]])

In [28]:
#Verificando dimensões do dataSet
print(dataSet.shape)

(13455, 11)


In [29]:
'''#Função para avaliar o desempenho da rede
def acerto(y_true, y_pred):
  try:
    print(y_true.eval())
    total = y_true.eval()
    acertos = 0

    for i in range(total):
      if (y_true[i] <= (y_pred[i] + 5)) and (y_true[i] >= (y_pred[i] - 5)):
        acertos += 1

    return (acertos / total)
  except:
    return 0
    '''

'#Função para avaliar o desempenho da rede\ndef acerto(y_true, y_pred):\n  try:\n    print(y_true.eval())\n    total = y_true.eval()\n    acertos = 0\n\n    for i in range(total):\n      if (y_true[i] <= (y_pred[i] + 5)) and (y_true[i] >= (y_pred[i] - 5)):\n        acertos += 1\n\n    return (acertos / total)\n  except:\n    return 0\n    '

In [0]:
#Definindo uma função para fazer o Cross Validation com a MLP
def trainMLP():    
  #Criando o modelo
  mlp = Sequential()
  #Adicionando a camada de entrada
  mlp.add(Dense(activation = 'tanh', input_dim = 11, units = 11, kernel_initializer = 'normal'))
  #Adicionando a primeira camada escondida
  mlp.add(Dense(activation = 'tanh', units = 11, kernel_initializer = 'normal'))
  #Adicionando a segunda camada escondida
  mlp.add(Dense(activation = 'tanh', units = 11, kernel_initializer='normal'))
  #Adicionando a camada de saída
  mlp.add(Dense(units = 1, kernel_initializer='normal'))    
  # Compilando o modelo
  mlp.compile(loss='mae', optimizer='rmsprop', metrics= ['mse', 'acc'])
  return mlp

In [0]:
#Modelo de classificação do Keras
estimator = KerasClassifier(build_fn=trainMLP, epochs=100, batch_size=50, verbose=1)

In [0]:
#Criando o objeto para Cross Validation
kfold = KFold(n_splits=4, shuffle=True, random_state=7)

In [33]:
#Treinando a rede
estimator.fit(dataSet, prec)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
13455/13455 [==============================] - 1s 61us/step - loss: 40.8172 - mean_squared_error: 11312.8030 - acc: 0.3597
Epoch 2/100
13455/13455 [==============================] - 0s 21us/step - loss: 40.5327 - mean_squared_error: 11137.4804 - acc: 0.1548
Epoch 3/100
13455/13455 [==============================] - 0s 22us/step - loss: 40.3341 - mean_squared_error: 11012.7579 - acc: 0.1271
Epoch 4/100
13455/13455 [==============================] - 0s 21us/step - loss: 40.1794 - mean_squared_error: 10900.5084 - acc: 0.1063
Epoch 5/100
13455/13455 [==============================] - 0s 21us/step - loss: 40.0410 - mean_squared_error: 10802.8019 - acc: 0.0946
Epoch 6/100
13455/13455 [==============================] - 0s 21us/step - loss: 39.9205 - mean_squared_error: 10713.5316 - acc: 0.0823
Epoch 7/100
13455/13455 [==============================] - 0s 21us/ste

In [0]:
#Treinando a rede
results = cross_val_score(estimator = estimator, X = dataSet, y = prec, cv=kfold, n_jobs = -1)

In [0]:
estimator.predict()

In [0]:
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150]
batches = [5, 10, 20]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))